In [ ]:
import requests, json
import pandas as pd
import datetime as dt

The Ontario Covid data shows the vacination status of incoming patient to ICU

In [ ]:
resource_id = '274b819c-5d69-4539-a4db-f2950794138c'
#limit the data to 100 to prevent massive data which cause computer issues. 
limit = '100'

api_url = 'https://data.ontario.ca/en/api/3/action/datastore_search?resource_id={}&limit={}'.format(resource_id, limit)
api_url

'https://data.ontario.ca/en/api/3/action/datastore_search?resource_id=274b819c-5d69-4539-a4db-f2950794138c&limit=100'

GET request

In [ ]:
data = requests.get(api_url)
data = data.json()

In [ ]:
data.keys()

dict_keys(['help', 'success', 'result'])

Extracting Values from API

In [ ]:
# displace the result & records
data['result']['records']

[{'_id': 1,
  'date': '2021-08-10T00:00:00',
  'icu_unvac': 22,
  'icu_partial_vac': 3,
  'icu_full_vac': 0,
  'hospitalnonicu_unvac': 23,
  'hospitalnonicu_partial_vac': 4,
  'hospitalnonicu_full_vac': 11},
 {'_id': 2,
  'date': '2021-08-11T00:00:00',
  'icu_unvac': 37,
  'icu_partial_vac': 5,
  'icu_full_vac': 2,
  'hospitalnonicu_unvac': 34,
  'hospitalnonicu_partial_vac': 7,
  'hospitalnonicu_full_vac': 8},
 {'_id': 3,
  'date': '2021-08-12T00:00:00',
  'icu_unvac': 45,
  'icu_partial_vac': 5,
  'icu_full_vac': 2,
  'hospitalnonicu_unvac': 44,
  'hospitalnonicu_partial_vac': 7,
  'hospitalnonicu_full_vac': 9},
 {'_id': 4,
  'date': '2021-08-13T00:00:00',
  'icu_unvac': 52,
  'icu_partial_vac': 5,
  'icu_full_vac': 3,
  'hospitalnonicu_unvac': 65,
  'hospitalnonicu_partial_vac': 6,
  'hospitalnonicu_full_vac': 8},
 {'_id': 5,
  'date': '2021-08-14T00:00:00',
  'icu_unvac': 53,
  'icu_partial_vac': 4,
  'icu_full_vac': 1,
  'hospitalnonicu_unvac': 67,
  'hospitalnonicu_partial_vac': 

Store the data into a DataFrame

In [ ]:
hos_by_vac = pd.DataFrame(data['result']['records'])
hos_by_vac

_id                 date  icu_unvac  icu_partial_vac  icu_full_vac  \
0     1  2021-08-10T00:00:00         22                3             0   
1     2  2021-08-11T00:00:00         37                5             2   
2     3  2021-08-12T00:00:00         45                5             2   
3     4  2021-08-13T00:00:00         52                5             3   
4     5  2021-08-14T00:00:00         53                4             1   
..  ...                  ...        ...              ...           ...   
95   96  2021-11-13T00:00:00         66                3            15   
96   97  2021-11-14T00:00:00         62                5            16   
97   98  2021-11-15T00:00:00         62                3            18   
98   99  2021-11-16T00:00:00         64                4            19   
99  100  2021-11-17T00:00:00         62                5            19   

    hospitalnonicu_unvac  hospitalnonicu_partial_vac  hospitalnonicu_full_vac  
0                     23                           4                       11  
1                     34                           7                        8  
2                     44                           7                        9  
3                     65                           6                        8  
4                     67                           6                       11  
..                   ...                         ...                      ...  
95                    81                          16                       74  
96                    87                          17                       78  
97                   102                          18                       81  
98                   102                          17                       76  
99                    92                          16                       70  

[100 rows x 8 columns]

Data Preparation for ML

In [ ]:
#check missing value
hos_by_vac.isna().sum() 
#the data has no missing value, so no further data cleaning is needed. 

_id                           0
date                          0
icu_unvac                     0
icu_partial_vac               0
icu_full_vac                  0
hospitalnonicu_unvac          0
hospitalnonicu_partial_vac    0
hospitalnonicu_full_vac       0
dtype: int64

In [ ]:
#convert time to datetime object
hos_by_vac['date'] = pd.to_datetime(hos_by_vac['date'])
#sum up the icu total visits
hos_by_vac['total_icu_visit'] = hos_by_vac['icu_unvac']+hos_by_vac['icu_partial_vac']+hos_by_vac['icu_full_vac']

hos_by_vac.head()

_id       date  icu_unvac  icu_partial_vac  icu_full_vac  \
0    1 2021-08-10         22                3             0   
1    2 2021-08-11         37                5             2   
2    3 2021-08-12         45                5             2   
3    4 2021-08-13         52                5             3   
4    5 2021-08-14         53                4             1   

   hospitalnonicu_unvac  hospitalnonicu_partial_vac  hospitalnonicu_full_vac  \
0                    23                           4                       11   
1                    34                           7                        8   
2                    44                           7                        9   
3                    65                           6                        8   
4                    67                           6                       11   

   total_icu_visit  
0               25  
1               44  
2               52  
3               60  
4               58

Data Cleaning and Feature Engineering

In [ ]:
# accumulate the total visit of icu since 2021/08/10
hos_by_vac['accu_icu_visits'] = 0
for i in range(len(hos_by_vac['date'])):
    if i == 0:
        hos_by_vac.loc[i, 'accu_icu_visits'] = hos_by_vac.loc[i, 'total_icu_visit']
    else:
        hos_by_vac.loc[i, 'accu_icu_visits'] = hos_by_vac.loc[i, 'total_icu_visit'] +\
        hos_by_vac.loc[i-1, 'total_icu_visit']
hos_by_vac

_id       date  icu_unvac  icu_partial_vac  icu_full_vac  \
0     1 2021-08-10         22                3             0   
1     2 2021-08-11         37                5             2   
2     3 2021-08-12         45                5             2   
3     4 2021-08-13         52                5             3   
4     5 2021-08-14         53                4             1   
..  ...        ...        ...              ...           ...   
95   96 2021-11-13         66                3            15   
96   97 2021-11-14         62                5            16   
97   98 2021-11-15         62                3            18   
98   99 2021-11-16         64                4            19   
99  100 2021-11-17         62                5            19   

    hospitalnonicu_unvac  hospitalnonicu_partial_vac  hospitalnonicu_full_vac  \
0                     23                           4                       11   
1                     34                           7                        8   
2                     44                           7                        9   
3                     65                           6                        8   
4                     67                           6                       11   
..                   ...                         ...                      ...   
95                    81                          16                       74   
96                    87                          17                       78   
97                   102                          18                       81   
98                   102                          17                       76   
99                    92                          16                       70   

    total_icu_visit  accu_icu_visits  
0                25               25  
1                44               69  
2                52               96  
3                60              112  
4                58              118  
..              ...              ...  
95               84              166  
96               83              167  
97               83              166  
98               87              170  
99               86              173  

[100 rows x 10 columns]

Establish Data Pipeline

In [ ]:
import sqlalchemy as sa
from sqlalchemy import create_engine, types

In [ ]:
# Write a function to request the data
# preparing the data importing to SQL database 
db_secret = {
    "drivername": "postgresql+psycopg2",
    "host": "______________.cloudapp.azure.com",
    "port": ____,
    "database": "hungchi",
    "user": "hungchi",
    "password": "_________",
}

def create_db_engine(**args):
    
    connection_url = sa.engine.URL.create(
        drivername = args.pop("drivername"),
        username   = args.pop("user"),
        password   = args.pop("password"),
        host       = args.pop("host"),
        port       = args.pop("port"),
        database   = args.pop("database") if 'database' in args else ''
    )
    
    engine = sa.create_engine(connection_url, **args)
    
    return engine

engine = create_db_engine(**db_secret)

In [ ]:
# building the connection
conection= engine.connect()
conection.execute('CREATE SCHEMA IF NOT EXISTS dimensions')
conection.close()

In [ ]:
# Ingest the data into  database in PostgreSQL server

#import data
hos_by_vac.to_sql(
    name= 'icu_cases', 
    con= engine,
    schema= 'dimensions', 
    if_exists= 'replace',
    index= False, 
    dtype= {
        'date':types.DATE, 
        'new_cases': types.DECIMAL(10,0)
    },
method= 'multi'


)